In [1]:
!pip install wandb
!pip install onnxruntime==1.9.0

  Using cached urllib3-1.26.8-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

conda 4.10.3 requires ruamel_yaml_conda>=0.11.14, which is not installed.
requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.8 which is incompatible.
dvc 2.5.4 requires ruamel.yaml>=0.16.1, but you'll have ruamel-yaml 0.15.87 which is incompatible.
bentoml 0.13.1 requires urllib3<=1.25.11, but you'll have urllib3 1.26.8 which is incompatible.


In [6]:
!pip install -q bentoml --use-feature=2020-resolver "skl2onnx==1.10.0" "onnx==1.10.1" "onnxmltools==1.10.0"

In [7]:
import os
import wandb
import pandas as pd
import numpy as np
import onnxruntime as rt

In [8]:
import onnxruntime

In [9]:
onnxruntime.__version__

'1.9.0'

## Download the model from Weights and Biases

In [10]:
import os
os.environ["WANDB_API_KEY"] = "2217d1580a9bb6478b3f7d6b4a501bb5a7f2d49d"

In [11]:
run = wandb.init(project='usedCarV2')

wandb: Currently logged in as: manaranjan (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [12]:
ARTIFACT_NAME = 'XGBoost_UserCar:v0'
#run = wandb.init()
artifact = run.use_artifact('manaranjan/usedCarV2/' + ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()
artifact_dir

'./artifacts/XGBoost_UserCar:v0'

In [13]:
!ls -al  artifacts/XGBoost_UserCar:v4

ls: artifacts/XGBoost_UserCar:v4: No such file or directory


In [14]:
%%writefile usedcar_price_prediction.py

import pandas as pd
import numpy as np
import bentoml
from bentoml.adapters import JsonInput
from bentoml.frameworks.onnx import OnnxModelArtifact

@bentoml.artifacts([OnnxModelArtifact('model')])
@bentoml.env(infer_pip_packages=True)
class UsedCarPriceOnnx(bentoml.BentoService):
    
    @bentoml.api(input=JsonInput(), batch=False)
    def predict(self, json):

      print(json)

      df = pd.DataFrame(json, index = [0])

      x_columns = ['KM_Driven', 'Fuel_Type', 'age',
                    'Transmission', 'Owner_Type', 'Seats',
                    'make', 'mileage_new', 'engine_new', 'model',
                    'power_new', 'Location']

      cat_features = ['Fuel_Type',
                      'Transmission', 'Owner_Type', 'model',
                      'make', 'Location']

      num_features = list(set(x_columns) - set(cat_features))       

      inputs = {c: df[c].values for c in df.columns}
      for c in num_features:
        inputs[c] = inputs[c].astype(np.float32)
      for k in inputs:
        inputs[k] = inputs[k].reshape((inputs[k].shape[0], 1))  

      outputs = self.artifacts.model.run(None, inputs)

      return f"Expected Sale Price of your car is: INR {np.round(float(outputs[0][0,0]), 1)} Lakhs"

Overwriting usedcar_price_prediction.py


In [15]:
import onnx

# onnx_model is an in-memory ModelProto
onnx_model = onnx.load(artifact_dir + "/usedcar_xgboost.onnx")

In [16]:
from usedcar_price_prediction import UsedCarPriceOnnx

In [17]:
svc = UsedCarPriceOnnx()
svc.pack('model', onnx_model)

saved_path = svc.save()

[2022-02-18 15:02:01,721] INFO - BentoService bundle 'UsedCarPriceOnnx:20220218150200_27AECF' saved to: /Users/manaranjan/bentoml/repository/UsedCarPriceOnnx/20220218150200_27AECF


In [19]:
!bentoml containerize UsedCarPriceOnnx:latest -t usedcarprice-mlops-18feb2022

[2022-02-18 15:22:40,725] INFO - Getting latest version UsedCarPriceOnnx:20220218150200_27AECF
Found Bento: /Users/manaranjan/bentoml/repository/UsedCarPriceOnnx/20220218150200_27AECF
Containerizing UsedCarPriceOnnx:20220218150200_27AECF with local YataiService and docker daemon from local environment/
Build container image: usedcarprice-mlops-18feb2022:20220218150200_27AECF


### Basic Docker Commands

1. Listing docker images

```
docker images
```
2.  Running the docker image

```
docker run -p 5000:5000 usedcarprice-mlopsv0 --workers=1
```

3. Go to browser and connect to the port

http://localhost:5000/

And then enter the following request in the request textbox

{"KM_Driven":10.0,"Fuel_Type":"Petrol","age":4.0,"Transmission":"Automatic","Owner_Type":"First","Seats":4.0,"make":"maruti","mileage_new":21.0,"engine_new":900.0,"model":"swift","power_new":90.0,"Location":"Bangalore"}

4. Deleting a docker image

In [20]:
import sklearn

In [21]:
sklearn.__version__

'1.0.2'

In [22]:
!conda info


     active environment : base
    active env location : /opt/anaconda3/envs/mlopslab
            shell level : 2
       user config file : /Users/manaranjan/.condarc
 populated config files : /Users/manaranjan/.condarc
          conda version : 4.10.3
    conda-build version : not installed
         python version : 3.8.5.final.0
       virtual packages : __osx=10.15.7=0
                          __unix=0=0
                          __archspec=1=x86_64
       base environment : /opt/anaconda3/envs/mlopslab  (writable)
      conda av data dir : /opt/anaconda3/envs/mlopslab/etc/conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/osx-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/osx-64
                          https://repo.anaconda.com/pkgs/r/noarch
          package cache : /opt/anaconda3/envs/mlopslab/pkgs
                          /Users/manaranjan/.con